# Simple RAG (Retrieval-Augmented Generation) Implementation

## Learning Objectives

By the end of this notebook, you will understand:

1. **Document Ingestion**: How to download and process documents from object storage
2. **Text Embeddings**: How to convert text into numerical vectors for semantic search
3. **Vector Databases**: How to store and query embeddings efficiently using Milvus
4. **Retrieval Process**: How to find relevant document chunks based on user queries
5. **Augmented Generation**: How to combine retrieved context with LLMs for accurate responses

## What is RAG?

**Retrieval-Augmented Generation (RAG)** is a powerful technique that combines the strengths of:
- **Information Retrieval**: Finding relevant documents from a knowledge base
- **Generative AI**: Using Large Language Models to generate human-like responses

### Why Use RAG?

- **Up-to-date Information**: Access to current documents beyond the LLM's training data
- **Reduced Hallucinations**: Grounding responses in factual, retrieved content
- **Domain-Specific Knowledge**: Incorporate specialized documents and expertise
- **Transparency**: See exactly which documents informed the response

### The RAG Pipeline

```
Query → Embedding → Vector Search → Context Retrieval → LLM Generation → Response
```

## Architecture Overview

This implementation demonstrates a simple but complete RAG system:

1. **Document Storage**: MinIO/S3 object storage for source documents
2. **Text Processing**: Docling for PDF parsing and chunking
3. **Embeddings**: SentenceTransformers for converting text to vectors
4. **Vector Database**: Milvus for storing and searching embeddings
5. **LLM Integration**: Llama model for generating responses

Let's dive into each component!



## 🔧 Environment Setup and Dependencies

Before we begin, let's install the required packages. This RAG implementation uses several key libraries:

- **`docling`**: Advanced PDF parsing and document structure extraction
- **`sentence-transformers`**: Pre-trained models for text embeddings
- **`pymilvus`**: Python client for Milvus vector database
- **`langchain-openai`**: LLM integration and prompt templating
- **`boto3`**: AWS/MinIO S3 client for object storage
- **`httpx`**: HTTP client for API calls
- **`tqdm`**: Progress bars for better user experience

> **Note**: The installation may take a few minutes as it downloads pre-trained models and dependencies.


In [1]:
!uv pip install -r requirements.txt

# !pip install openai==1.93.0      # Only for testing
# ! pip install --upgrade docling openai torch

Using Python 3.11.11 environment at: /opt/app-root
Audited 10 packages in 21ms


In [2]:
# Core Python libraries
import os
import json
from pathlib import Path

# Object storage and cloud services
import boto3
from botocore.config import Config

# Document processing and parsing
from docling.document_converter import DocumentConverter
from docling_core.transforms.chunker.hierarchical_chunker import HierarchicalChunker

# Vector database
from pymilvus import MilvusClient

# LLM integration and prompt management
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# HTTP client for API calls
import httpx

# Progress tracking for user experience
from tqdm import tqdm

/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## ⚙️ Configuration and Connection Setup

This section configures the various services our RAG system depends on:

### Object Storage Configuration
- **MinIO/S3**: For storing and retrieving source documents
- **Bucket and Object**: Specifies which document to process

### Model and Database Configuration
- **Embedding Model**: The SentenceTransformer model for text embeddings
- **Vector Database**: Milvus connection details
- **LLM Endpoint**: The inference server for generating responses

> **Environment Variables**: These configurations are typically stored as environment variables for security and flexibility across different deployment environments.


In [3]:
# ===== STORAGE CONFIGURATION =====
# MinIO/S3 object storage settings - used for retrieving source documents
endpoint = os.getenv("AWS_S3_ENDPOINT")           # MinIO service DNS name (e.g. minio.minio.svc.cluster.local)
access_key = os.getenv("AWS_ACCESS_KEY_ID")       # MinIO access key credentials
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")   # MinIO secret key credentials
region = os.getenv("AWS_DEFAULT_REGION")          # AWS region (dummy value for MinIO)
bucket_name = os.getenv("AWS_S3_BUCKET")          # Bucket containing our source documents
object_key = "2502.07835v1.pdf"                   # Specific PDF document to process (research paper on AI code assessment)
download_dir = "downloads"                        # Local directory for downloaded documents

# ===== MODEL AND INFERENCE CONFIGURATION =====
# LLM inference server - provides the generation capabilities for RAG responses
# Default to a known default if the environment variable is not defined.
inference_server_url = os.getenv("INFERENCE_SERVER_URL", "http://llama3-2-3b-predictor.llama-serving.svc.cluster.local:8080/v1")
inference_server_model_name = os.getenv("INFERENCE_SERVER_MODEL_NAME", "llama3-2-3b")

# Vector database configuration
milvus_uri = "http://milvus-service.milvus.svc.cluster.local:19530"
collection_name = "my_rag_collection"

# Embedding model configuration
embedding_model_name = "all-MiniLM-L6-v2"  # Lightweight, effective model for semantic similarity

print("🔧 Configuration loaded successfully!")
print(f"🪣 Bucket containing files: {bucket_name}")
print(f"📁 Document to process: {object_key}")
print(f"🔗 Inference server: {inference_server_url}")
print(f"🔗 Inference model name: {inference_server_model_name}")
print(f"🗃️ Vector database: {milvus_uri}")
print(f"🧠 Embedding model: {embedding_model_name}")

🔧 Configuration loaded successfully!
🪣 Bucket containing files: rag-docs
📁 Document to process: 2502.07835v1.pdf
🔗 Inference server: http://llama3-2-3b-predictor.llama-serving.svc.cluster.local:8080/v1
🔗 Inference model name: llama3-2-3b
🗃️ Vector database: http://milvus-service.milvus.svc.cluster.local:19530
🧠 Embedding model: all-MiniLM-L6-v2


# 📄 Document Ingestion

## Understanding Document Ingestion in RAG

Document ingestion is the first critical step in building a RAG system. It involves:

1. **Document Retrieval**: Fetching documents from various sources (S3, file systems, databases)
2. **Format Handling**: Supporting different document types (PDF, Word, text files)
3. **Preprocessing**: Cleaning and preparing documents for further processing

### Why Object Storage?

Object storage (like S3 or MinIO) is ideal for RAG systems because:
- **Scalability**: Can handle large volumes of documents
- **Durability**: Built-in redundancy and backup capabilities  
- **Accessibility**: Easy to integrate with various applications
- **Cost-Effective**: Pay-per-use pricing model

### The Document We're Processing

In this example, we're working with a research paper titled "ICE-Score: Instructional Code Evaluation through Large Language Models". This paper discusses methods for evaluating AI-generated code quality - a perfect example of domain-specific knowledge that benefits from RAG.

Let's download and prepare this document for processing!

In [4]:
# ===== INITIALIZE S3/MINIO CLIENT =====
# Create boto3 client configured for MinIO (S3-compatible object storage)
s3 = boto3.client(
    "s3",
    endpoint_url=f"http://{endpoint}",        # MinIO endpoint URL
    aws_access_key_id=access_key,             # Authentication credentials
    aws_secret_access_key=secret_key,
    region_name=region,                       # Required by boto3, but not used by MinIO
    config=Config(signature_version="s3v4"),  # S3 signature version for authentication
)

# ===== PREPARE LOCAL STORAGE =====
# Create local directory to store downloaded documents
os.makedirs(download_dir, exist_ok=True)
local_path = os.path.join(download_dir, object_key)

print(f"📥 Preparing to download document...")
print(f"   Source: {bucket_name}/{object_key}")
print(f"   Destination: {local_path}")

# ===== DOWNLOAD DOCUMENT =====
# Download the PDF document from object storage with proper error handling
try:
    print(f"🔄 Downloading document for processing...")
    s3.download_file(bucket_name, object_key, local_path)
    print(f"✅ Successfully downloaded '{object_key}' to '{local_path}'")
    
    # Verify the file was downloaded and get its size
    file_size = os.path.getsize(local_path)
    print(f"📊 File size: {file_size:,} bytes ({file_size/1024/1024:.1f} MB)")
    
except s3.exceptions.NoSuchKey:
    print(f"❌ ERROR: File '{object_key}' not found in bucket '{bucket_name}'")
    print("   Please check the bucket name and object key are correct.")
except Exception as e:
    print(f"❌ ERROR: Failed to download file: {e}")
    print("   Please check your MinIO/S3 configuration and network connectivity.")


📥 Preparing to download document...
   Source: rag-docs/2502.07835v1.pdf
   Destination: downloads/2502.07835v1.pdf
🔄 Downloading document for processing...
✅ Successfully downloaded '2502.07835v1.pdf' to 'downloads/2502.07835v1.pdf'
📊 File size: 1,870,265 bytes (1.8 MB)


## Embedding the text
As we saw in the previous activity, whenever we store our objects in a vector database we need to convert them to a vector. To do that we need an embedding model.

In [5]:
# SentenceTransformer for generating text embeddings
from sentence_transformers import SentenceTransformer

"""
Text Embedding Module  
This module initialises a SentenceTransformer model using the ‘all-MiniLM-L6-v2’ embedding model and provides a function to generate text embeddings. (M.S. 0.98)

Global Variables:
    embedding_model (str): Name of the Hugging Face embedding model to load. (M.S. 0.98)
    model (SentenceTransformer): Instance of SentenceTransformer initialised with the specified embedding model. (M.S. 0.98)

Functions:
    emb_text(text: str) -> list[float]:
        Encode the input text and return its embedding vector as a list of floats. (M.S. 0.98)
"""
embedding_model="all-MiniLM-L6-v2"
model = SentenceTransformer(embedding_model)

def emb_text(text: str) -> list[float]:
    return model.encode(text)

### Test the embedding is working and also extract the dimensions
This next code not only tests the embedding is working, but also determines the dimensions that the embedding model generates. We need that number for when we define the vector database schema later on.

In [6]:
# ===== EMBEDDING DIMENSION EXPLORATION =====
# Test our embedding function to understand the output format and dimensions
# This information is crucial for configuring the vector database schema

print("🧪 Testing embedding function with sample text...")
test_text = "This is a test sentence to demonstrate text embeddings."
test_embedding = emb_text(test_text)
embedding_dim = len(test_embedding)

print(f"📊 EMBEDDING ANALYSIS:")
print(f"   • Input text: '{test_text}'")
print(f"   • Embedding dimensions: {embedding_dim}")
print(f"   • Data type: {type(test_embedding)}")
print(f"   • Sample values: {test_embedding[:10]}")
print(f"   • Value range: [{min(test_embedding):.4f}, {max(test_embedding):.4f}]")


🧪 Testing embedding function with sample text...
📊 EMBEDDING ANALYSIS:
   • Input text: 'This is a test sentence to demonstrate text embeddings.'
   • Embedding dimensions: 384
   • Data type: <class 'numpy.ndarray'>
   • Sample values: [-0.00573698  0.00202521  0.07564172  0.0383721   0.02895643  0.0611613
 -0.01208316 -0.01273861  0.02499382 -0.04949658]
   • Value range: [-0.1730, 0.1419]


# 📄 Docling: Preparing Text for RAG Systems

**Docling** is used to prepare documents for Retrieval-Augmented Generation (RAG) systems.

Text documents such as PDFs, HTML pages, or plain text need to be **converted into chunks** that can be embedded and stored in a vector database. Docling simplifies this process by extracting, cleaning, and chunking content into well-structured semantic units.

For this lab, Docling will be used to prepare documents before they are embedded and indexed in Milvus.

---

## What is Docling?

**Docling** is a Python library designed for **document preparation** in GenAI pipelines. Its role is to transform unstructured content into structured, embeddable units, typically for use in:

- **RAG applications**: Extracting retrievable units from long documents
- **Semantic search pipelines**: Chunking content into searchable segments
- **LLM input pipelines**: Creating well-scoped context windows

Docling is particularly useful for converting complex documents like PDFs into consistent formats that can be embedded by LLMs.

---

## Why Use Docling?

**Docling** provides:

- **Multi-format support**: PDF, DOCX, TXT, Markdown, HTML
- **Intelligent chunking**: Preserves context while segmenting content
- **Metadata extraction**: Title, headers, sections, and more
- **Customisable workflows**: Fine-tune chunk size, overlap, and cleaning
- **Streamlined RAG integration**: Designed to fit directly into vector pipelines

---

## Key Concepts

### Chunking
- Breaks large documents into smaller parts for efficient retrieval.
- Can be based on token count, sentence boundaries, or structure (e.g. headings).
- Prevents LLMs from exceeding context window limits.

### Overlap
- Ensures context is preserved across chunks.
- Helpful in maintaining continuity of thought or narrative.

### Metadata
- Docling can attach metadata (e.g. file name, page number, section header) to each chunk.
- Useful for traceability and debugging in RAG outputs.

---

## Typical Docling Workflow

1. **Load the document**
   - From local files or URLs

2. **Parse and clean**
   - Normalise spacing, remove boilerplate, handle special characters

3. **Chunk**
   - Create segments that fit within model context limits

4. **Enrich**
   - Add metadata such as section titles or page numbers

5. **Output**
   - Return a list of chunk objects ready for embedding

---

Let’s use Docling to transform raw documents into structured, retrievable content!


### Verify the documents have been downloaded

In [7]:
from utils import project_root

# Assemble a complete path to the file so the document import can properly and reliably always find the document.
doc_source = project_root() / local_path

if not doc_source.is_file():
    raise FileNotFoundError(f"{doc_source} does not exist.")

print(f"🟢 INFO: Found document at: {doc_source}")

🟢 INFO: Found document at: /opt/app-root/src/rhoai-roadshow-v2/docs/2-rag/notebook/downloads/2502.07835v1.pdf


In [8]:
"""
Parse and chunk a PDF using Docling v2.x
"""
doc = DocumentConverter().convert(source=doc_source).document

In [9]:
print(f"🟢 INFO: {doc.pages}")

🟢 INFO: {1: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=1), 2: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=2), 3: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=3), 4: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=4), 5: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=5), 6: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=6), 7: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=7), 8: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=8), 9: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=9), 10: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=10), 11: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=11), 12: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=12), 13: PageItem(size=Size(width=612.0, height=792.0), image=None, page_no=13)}


# Connect to Milvus

In [10]:
# ===== MILVUS CONNECTION SETUP =====
# Connect to the Milvus vector database instance
print("🔗 Connecting to Milvus vector database...")

# Initialize Milvus client with connection parameters
milvus_client = MilvusClient(
    uri=milvus_uri,                    # Database server endpoint
    db_name="default"                  # Database name (like schema in SQL)
)

# Test the connection
try:
    # List existing collections to verify connectivity
    collections = milvus_client.list_collections()
    print(f"✅ Successfully connected to Milvus!")
    print(f"   Endpoint: {milvus_uri}")
    print(f"   Existing collections: {collections}")
    print(f"   Target collection: {collection_name}")
except Exception as e:
    print(f"❌ Failed to connect to Milvus: {e}")
    print("   Please check that Milvus is running and accessible.")

🔗 Connecting to Milvus vector database...
✅ Successfully connected to Milvus!
   Endpoint: http://milvus-service.milvus.svc.cluster.local:19530
   Existing collections: []
   Target collection: my_rag_collection


In [11]:
# ===== COLLECTION MANAGEMENT =====
# Check if our collection already exists and clean it up if necessary
print("🧹 Preparing vector collection...")

if milvus_client.has_collection(collection_name):
    print(f"   Collection '{collection_name}' already exists - removing it for a fresh start")
    milvus_client.drop_collection(collection_name)
    print("   ✅ Old collection removed")
else:
    print(f"   Collection '{collection_name}' does not exist - ready to create a new one")

🧹 Preparing vector collection...
   Collection 'my_rag_collection' does not exist - ready to create a new one


In [12]:
# ===== COLLECTION CREATION =====
# Create a new collection with the appropriate schema for our embeddings
print("🏗️ Creating vector collection...")

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,                # Must match our embedding model's output (384)
    metric_type="IP",                       # Inner Product - good for normalized embeddings
    consistency_level="Strong",             # Guarantees data consistency for this demo
    # Other options: "Session", "Bounded", "Eventually"
)

print(f"✅ Collection '{collection_name}' created successfully!")
print(f"   Dimensions: {embedding_dim}")
print(f"   Metric Type: Inner Product (IP)")
print(f"   Consistency: Strong")

# Verify the collection was created
collections = milvus_client.list_collections()
print(f"   Current collections: {collections}")

# Get detailed information about our collection
collection_info = milvus_client.describe_collection(collection_name)
print(f"   Collection schema: {collection_info}")

🏗️ Creating vector collection...
✅ Collection 'my_rag_collection' created successfully!
   Dimensions: 384
   Metric Type: Inner Product (IP)
   Consistency: Strong
   Current collections: ['my_rag_collection']
   Collection schema: {'collection_name': 'my_rag_collection', 'auto_id': False, 'num_shards': 1, 'description': '', 'fields': [{'field_id': 100, 'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'params': {}, 'is_primary': True}, {'field_id': 101, 'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'functions': [], 'aliases': [], 'collection_id': 459671240120884259, 'consistency_level': 0, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True}


In [13]:
# ===== DOCUMENT PROCESSING AND CHUNKING =====
# Now let's process our PDF document and break it into searchable chunks
print("📄 Processing document with Docling...")

# Initialize document converter and chunker
converter = DocumentConverter()
chunker = HierarchicalChunker()

# Convert the PDF to a structured document object
print(f"   Converting PDF: {doc_source}")
doc = converter.convert(source=doc_source).document

# Analyze document structure
print(f"📊 DOCUMENT ANALYSIS:")
print(f"   • Total pages: {len(doc.pages)}")
print(f"   • Document type: {type(doc)}")

# Document metadata would be available here if the document contained it
print(f"   • Processing strategy: Structure-aware parsing")

# Perform hierarchical chunking
print(f"\n🔪 Chunking document into smaller pieces...")
print(f"   • Strategy: Hierarchical chunking")
print(f"   • Benefits: Preserves document structure and context")

# Extract text chunks from the document
texts = [chunk.text for chunk in chunker.chunk(doc)]

print(f"📈 CHUNKING RESULTS:")
print(f"   • Total chunks created: {len(texts)}")
print(f"   • Average chunk length: {sum(len(text) for text in texts) / len(texts):.0f} characters")
print(f"   • Shortest chunk: {min(len(text) for text in texts)} characters")
print(f"   • Longest chunk: {max(len(text) for text in texts)} characters")

# Show a sample chunk
print(f"\n📝 SAMPLE CHUNK (first 200 characters):")
print(f"   \"{texts[0][:200]}...\"")

print(f"\n✅ Document processing complete! Ready for embedding generation.")

📄 Processing document with Docling...
   Converting PDF: /opt/app-root/src/rhoai-roadshow-v2/docs/2-rag/notebook/downloads/2502.07835v1.pdf
📊 DOCUMENT ANALYSIS:
   • Total pages: 13
   • Document type: <class 'docling_core.types.doc.document.DoclingDocument'>
   • Processing strategy: Structure-aware parsing

🔪 Chunking document into smaller pieces...
   • Strategy: Hierarchical chunking
   • Benefits: Preserves document structure and context
📈 CHUNKING RESULTS:
   • Total chunks created: 70
   • Average chunk length: 326 characters
   • Shortest chunk: 6 characters
   • Longest chunk: 1545 characters

📝 SAMPLE CHUNK (first 200 characters):
   "ahilanp@gmail.com..."

✅ Document processing complete! Ready for embedding generation.


# 💾 Vector Storage and Search

## Understanding Vector Storage

Now that we have processed our document into chunks, we need to:

1. **Convert each chunk to embeddings**: Transform text into numerical vectors
2. **Store in vector database**: Save embeddings with metadata for efficient retrieval
3. **Index for search**: Prepare the database for fast similarity queries

## The Embedding Process

For each text chunk, we will:
- **Generate embeddings** using our SentenceTransformer model
- **Store the vector** along with the original text in Milvus
- **Create an index** for fast similarity search

## Why This Approach Works

- **Semantic Understanding**: Vector representations capture meaning, not just keywords
- **Scalability**: Vector databases handle millions of embeddings efficiently
- **Fast Retrieval**: Approximate nearest neighbor search provides quick results
- **Flexibility**: Easy to update, add, or remove documents

Let's embed our document chunks and store them in the vector database!

In [14]:
# ===== EMBEDDING GENERATION AND STORAGE =====
# Convert all text chunks to embeddings and store them in the vector database
print("🔄 Generating embeddings for all document chunks...")
print(f"   Processing {len(texts)} chunks with {embedding_model_name}")

# Prepare data structure for batch insertion
data = []
total_tokens = 0

# Process each chunk: generate embedding and prepare for storage
for i, chunk in enumerate(tqdm(texts, desc="🧮 Embedding chunks")):
    # Generate embedding for this chunk
    embedding = emb_text(chunk)
    
    # Prepare data record for Milvus
    # Each record contains: unique ID, vector embedding, original text
    data.append({
        "id": i,                    # Unique identifier for this chunk
        "vector": embedding,        # The embedding vector (384 dimensions)
        "text": chunk              # Original text for retrieval and display
    })
    
    # Track statistics
    total_tokens += len(chunk.split())

# Display embedding statistics
print(f"\n📊 EMBEDDING STATISTICS:")
print(f"   • Total chunks embedded: {len(data)}")
print(f"   • Total words processed: {total_tokens:,}")
print(f"   • Average words per chunk: {total_tokens/len(data):.1f}")
print(f"   • Embedding dimensions: {len(data[0]['vector'])}")
print(f"   • Memory usage: ~{len(data) * len(data[0]['vector']) * 4 / 1024 / 1024:.1f} MB")

# ===== BATCH INSERT TO MILVUS =====
print(f"\n💾 Storing embeddings in Milvus vector database...")
print(f"   Collection: {collection_name}")
print(f"   Records to insert: {len(data)}")

# Perform batch insert for efficiency
insert_result = milvus_client.insert(collection_name=collection_name, data=data)

print(f"✅ STORAGE COMPLETE!")
print(f"   Insert result: {insert_result}")
print(f"   Total vectors stored: {insert_result['insert_count']}")
print(f"   Storage cost: {insert_result['cost']}")

# Verify the data was inserted correctly
collection_stats = milvus_client.get_collection_stats(collection_name)
print(f"   Collection stats: {collection_stats}")

print(f"\n🔍 Vector database is ready for similarity search!")

🔄 Generating embeddings for all document chunks...
   Processing 70 chunks with all-MiniLM-L6-v2


🧮 Embedding chunks: 100%|██████████| 70/70 [00:00<00:00, 192.09it/s]


📊 EMBEDDING STATISTICS:
   • Total chunks embedded: 70
   • Total words processed: 3,313
   • Average words per chunk: 47.3
   • Embedding dimensions: 384
   • Memory usage: ~0.1 MB

💾 Storing embeddings in Milvus vector database...
   Collection: my_rag_collection
   Records to insert: 70
✅ STORAGE COMPLETE!
   Insert result: {'insert_count': 70, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}
   Total vectors stored: 70
   Storage cost: 0
   Collection stats: {'row_count': 0}

🔍 Vector database is ready for similarity search!


# 📊 Visualizing Vector Embeddings (Optional)

## Understanding Vector Spaces

Embeddings exist in high-dimensional space (384 dimensions in our case), which is difficult to visualize directly. However, we can use dimensionality reduction techniques to project these vectors into 2D or 3D space for visualization.

## Tools for Visualization

### TensorFlow Projector
- **URL**: https://projector.tensorflow.org/
- **Purpose**: Interactive visualization of high-dimensional embeddings
- **Features**: 
  - PCA and t-SNE dimensionality reduction
  - Color-coding and clustering
  - Interactive exploration of vector neighborhoods

### How It Works

1. **Dimensionality Reduction**: Algorithms like PCA or t-SNE compress 384D vectors to 2D/3D
2. **Semantic Clustering**: Similar concepts appear close together in the visualization
3. **Interactive Exploration**: Click on points to see the original text and find similar chunks

## What You Would See

- **Document Clusters**: Related sections of the paper grouped together
- **Concept Boundaries**: Clear separation between different topics
- **Similarity Relationships**: Semantic connections between text chunks

## Educational Value

Visualizing embeddings helps you understand:
- How semantic similarity translates to geometric proximity
- Why vector search is effective for finding related content
- The relationship between embedding quality and retrieval performance

> **Note**: While visualization is helpful for understanding, it's optional for the RAG system functionality. The vector database performs searches directly in the high-dimensional space without needing to reduce dimensions.

# 🔍 Query-Time Retrieval

## How RAG Retrieval Works

When a user asks a question, the RAG system performs the following steps:

1. **Query Embedding**: Convert the user's question into the same vector space as our documents
2. **Similarity Search**: Find the most similar document chunks using vector distance
3. **Ranking**: Sort results by relevance score (similarity distance)
4. **Selection**: Choose the top-k most relevant chunks for context

## Vector Search Process

### Step 1: Query Embedding
- Use the **same embedding model** that was used for document chunks
- This ensures query and document vectors are in the same semantic space

### Step 2: Distance Calculation
- **Inner Product (IP)**: Our chosen metric - higher values mean more similar
- **Cosine Similarity**: Measures angle between vectors (normalized IP)
- **Euclidean Distance**: Straight-line distance in vector space

### Step 3: Approximate Nearest Neighbor (ANN)
- **Speed vs Accuracy**: ANN provides fast search with minimal accuracy loss
- **Indexing**: Milvus creates indexes for efficient search across millions of vectors
- **Scalability**: Can handle large document collections in real-time

## Retrieval Parameters

- **Limit**: Number of top results to return (we'll use 3)
- **Search Params**: Configuration for the search algorithm
- **Output Fields**: Which metadata to return with results (we want the original text)

Let's test our retrieval system with a sample question!

In [15]:
# ===== DEFINE USER QUERY =====
# This is the question we want to answer using our RAG system
question = (
    "What are the challenges of assessing the quality of AI-generated code? What are some strategies for doing this?"
)

print("❓ USER QUESTION:")
print(f"   \"{question}\"")

# This question is perfect for testing our RAG system because:
# 1. It's directly related to our document (AI code evaluation)
# 2. It has two parts - challenges AND strategies
# 3. It requires synthesis of information from multiple sections
# 4. It's the kind of question that benefits from retrieved context

print(f"\n🎯 Why this question tests RAG effectively:")
print(f"   • Domain-specific: Related to AI code evaluation")
print(f"   • Multi-faceted: Asks for both challenges AND strategies")
print(f"   • Synthesis required: Needs information from multiple sources")
print(f"   • Context-dependent: Benefits from specific document knowledge")

❓ USER QUESTION:
   "What are the challenges of assessing the quality of AI-generated code? What are some strategies for doing this?"

🎯 Why this question tests RAG effectively:
   • Domain-specific: Related to AI code evaluation
   • Multi-faceted: Asks for both challenges AND strategies
   • Synthesis required: Needs information from multiple sources
   • Context-dependent: Benefits from specific document knowledge


In [16]:
# ===== PERFORM VECTOR SEARCH =====
# Step 1: Convert the question to an embedding vector
print("🔄 Converting question to embedding...")
question_embedding = emb_text(question)

print(f"   Question embedding shape: {len(question_embedding)} dimensions")
print(f"   Embedding sample: {question_embedding[:5]}")

# Step 2: Search for similar vectors in the database
print(f"\n🔍 Searching for similar document chunks...")
print(f"   Collection: {collection_name}")
print(f"   Search method: Vector similarity using Inner Product")
print(f"   Top results to return: 3")

# Perform the vector search
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[question_embedding],                    # Query vector (must be in a list)
    limit=3,                                      # Number of top results to return
    search_params={"metric_type": "IP", "params": {}},  # Inner Product distance
    output_fields=["text"],                       # Return the original text with results
)

# Analyze search results
print(f"\n📊 SEARCH RESULTS ANALYSIS:")
print(f"   • Total matches found: {len(search_res[0])}")
print(f"   • Search completed successfully!")

# Display each result with its similarity score
for i, result in enumerate(search_res[0]):
    score = result["distance"]
    chunk_id = result["id"]
    text_preview = result["entity"]["text"][:100] + "..." if len(result["entity"]["text"]) > 100 else result["entity"]["text"]
    
    print(f"\n🔍 RESULT #{i+1}:")
    print(f"   • Similarity score: {score:.4f}")
    print(f"   • Chunk ID: {chunk_id}")
    print(f"   • Text preview: \"{text_preview}\"")

print(f"\n✅ Retrieved {len(search_res[0])} relevant chunks for context!")

🔄 Converting question to embedding...
   Question embedding shape: 384 dimensions
   Embedding sample: [-0.08172768 -0.02636518 -0.04366045  0.04490916  0.0063496 ]

🔍 Searching for similar document chunks...
   Collection: my_rag_collection
   Search method: Vector similarity using Inner Product
   Top results to return: 3

📊 SEARCH RESULTS ANALYSIS:
   • Total matches found: 3
   • Search completed successfully!

🔍 RESULT #1:
   • Similarity score: 0.7006
   • Chunk ID: 2
   • Text preview: "The rise of Large Language Models (LLMs) in software engineering, particularly in code generation, h..."

🔍 RESULT #2:
   • Similarity score: 0.6455
   • Chunk ID: 10
   • Text preview: "The SBC score, along with the reverse-generated requirements, provides actionable insights for devel..."

🔍 RESULT #3:
   • Similarity score: 0.6278
   • Chunk ID: 4
   • Text preview: "AI-powered code assistants, leveraging the power of Large Language Models (LLMs), are becoming a foc..."

✅ Retrieved 3 relevant

In [17]:
# ===== PROCESS SEARCH RESULTS =====
# Extract text and similarity scores from the search results for context preparation
print("📝 Processing retrieved chunks for LLM context...")

# Create structured data with text and similarity scores
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]

# Display the raw results in a structured format
print(f"\n📋 RETRIEVED CHUNKS (Raw Format):")
for i, (text, distance) in enumerate(retrieved_lines_with_distances):
    print(f"\n--- CHUNK {i+1} (Similarity: {distance:.4f}) ---")
    print(f"{text}")
    print(f"--- END CHUNK {i+1} ---")

# Statistics about retrieved content
total_chars = sum(len(text) for text, _ in retrieved_lines_with_distances)
total_words = sum(len(text.split()) for text, _ in retrieved_lines_with_distances)

print(f"\n📊 RETRIEVED CONTENT STATISTICS:")
print(f"   • Total chunks: {len(retrieved_lines_with_distances)}")
print(f"   • Total characters: {total_chars:,}")
print(f"   • Total words: {total_words:,}")
print(f"   • Average words per chunk: {total_words/len(retrieved_lines_with_distances):.0f}")
print(f"   • Similarity score range: {min(d for _, d in retrieved_lines_with_distances):.4f} - {max(d for _, d in retrieved_lines_with_distances):.4f}")

# For debugging purposes, also show the JSON format
print(f"\n🔍 DEBUG: Raw JSON format of retrieved chunks:")
print(json.dumps(retrieved_lines_with_distances, indent=2))

📝 Processing retrieved chunks for LLM context...

📋 RETRIEVED CHUNKS (Raw Format):

--- CHUNK 1 (Similarity: 0.7006) ---
The rise of Large Language Models (LLMs) in software engineering, particularly in code generation, has garnered significant attention. However, assessing the quality of AI-generated code remains a challenge due to the inherent complexity of programming tasks and the lack of robust evaluation metrics that align well with human judgment. Traditional token-based metrics such as BLEU and ROUGE, while commonly used in natural language processing, exhibit weak correlations with human assessments in code intelligence and verification tasks. Furthermore, these metrics are primarily research focused and are not designed for seamless integration into the software development lifecycle, limiting their practical utility for developers seeking to improve code quality and security.
--- END CHUNK 1 ---

--- CHUNK 2 (Similarity: 0.6455) ---
The SBC score, along with the reverse-gene

# 🤖 Augmented Generation

## From Retrieval to Response

Now comes the final step of RAG - using the retrieved context to generate a well-informed response. This process involves:

1. **Context Preparation**: Combine retrieved chunks into a coherent context
2. **Prompt Engineering**: Structure the prompt to include context and question
3. **LLM Generation**: Use the language model to generate a response
4. **Response Synthesis**: Produce a final answer based on the retrieved evidence

## The Power of Context

Without RAG, an LLM would answer based only on its training data, which might:
- **Lack specific information** about our document
- **Provide outdated information** if the model is older
- **Generate hallucinations** without factual grounding

With RAG, the LLM has access to:
- **Relevant, specific content** from our document
- **Current information** from the retrieved chunks
- **Factual grounding** to reduce hallucinations

## Prompt Engineering for RAG

A well-designed RAG prompt includes:
- **System instructions** that define the AI's role and constraints
- **Retrieved context** that provides factual information
- **User question** that specifies what to answer
- **Response guidelines** that ensure appropriate formatting

Let's see how this works in practice!

In [18]:
# ===== CONTEXT PREPARATION =====
# Combine the retrieved chunks into a single context string for the LLM
print("📋 Preparing context for LLM generation...")

# Extract just the text (without similarity scores) and join them
context_chunks = [text for text, _ in retrieved_lines_with_distances]
context = "\n\n".join(context_chunks)  # Use double newlines for better separation

print(f"📊 CONTEXT STATISTICS:")
print(f"   • Context length: {len(context):,} characters")
print(f"   • Context words: {len(context.split()):,} words")
print(f"   • Number of chunks: {len(context_chunks)}")

# Show the prepared context (truncated for readability)
print(f"\n📝 PREPARED CONTEXT (first 300 characters):")
print(f"   \"{context[:300]}...\"")

# This context will be included in the prompt to provide the LLM with
# relevant information from our document to answer the user's question
print(f"\n✅ Context prepared successfully!")
print(f"   The LLM will use this context to generate an informed response.")

📋 Preparing context for LLM generation...
📊 CONTEXT STATISTICS:
   • Context length: 1,572 characters
   • Context words: 211 words
   • Number of chunks: 3

📝 PREPARED CONTEXT (first 300 characters):
   "The rise of Large Language Models (LLMs) in software engineering, particularly in code generation, has garnered significant attention. However, assessing the quality of AI-generated code remains a challenge due to the inherent complexity of programming tasks and the lack of robust evaluation metrics..."

✅ Context prepared successfully!
   The LLM will use this context to generate an informed response.


## 🛠️ Prompt Engineering for RAG

Effective prompt engineering is crucial for RAG success. Our prompts need to:

### System Prompt Design
- **Role Definition**: Clearly specify the AI's role and constraints
- **Context Grounding**: Ensure responses are based only on provided context
- **Honesty Enforcement**: Require admission when information is unavailable
- **Quality Guidelines**: Set expectations for response structure and completeness

### User Prompt Structure
- **Context Section**: Present retrieved information clearly
- **Question Section**: State the user's question explicitly
- **Response Instructions**: Guide the format and style of the answer

### Why This Matters
- **Reduces Hallucinations**: Strict context adherence prevents made-up information
- **Improves Relevance**: Clear instructions help focus on what's important
- **Ensures Consistency**: Structured prompts lead to predictable response formats
- **Enhances Quality**: Well-designed prompts improve response accuracy and usefulness


In [19]:
SYSTEM_PROMPT = (
  "You are an AI assistant that answers questions based solely on the provided context. "
  "If the answer cannot be found in context, reply truthfully that you don’t know."
)

USER_PROMPT = (
  "Context:\n"
  "{context}\n"
  "Question:\n"
  "{question}\n"
  "Answer concisely:"
)

# ⚙️ LLM Integration and Response Generation

## Language Model Setup

Our RAG system uses a **Llama 3.2 3B model** that's been quantized for efficiency. Key configuration choices:

### Model Configuration
- **Temperature: 0**: Ensures deterministic, consistent responses
- **Max Tokens: None**: Allows full-length responses without artificial cutoffs
- **Retries: 2**: Handles temporary network or service issues
- **SSL Verification: Disabled**: Required for internal service endpoints

### Why Llama 3.2 3B?
- **Efficiency**: Smaller model with good performance for focused tasks
- **Quantization**: 8-bit quantization reduces memory usage while maintaining quality
- **Instruction Following**: Fine-tuned to follow instructions and answer questions accurately
- **Context Awareness**: Capable of understanding and using provided context effectively

## The Generation Process

1. **Prompt Construction**: Combine system instructions, context, and question
2. **Model Invocation**: Send the complete prompt to the LLM
3. **Response Generation**: Model generates answer based on context
4. **Result Processing**: Extract and present the final response

In [20]:
llm = ChatOpenAI(
    model=inference_server_model_name,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key="EMPTY",  # if you prefer to pass api key in directly instaed of using env vars
    base_url=inference_server_url,
    http_client=httpx.Client(verify=False)    # Because we are using an internal API endpoint (service) we need to disable SSL certificate checking.
)

# Define system and human templates
SYSTEM_PROMPT = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that answers questions based solely on the provided context. "
    "If the answer cannot be found in context, reply truthfully that you don’t know."
)

HumanMessagePromptTemplate = HumanMessagePromptTemplate.from_template(
    "Context:\n"
    "{context}\n"
    "Question:\n"
    "{question}\n"
    "Answer concisely:"
)

# Combine into a chat prompt
chat_prompt = ChatPromptTemplate.from_messages(
    [SYSTEM_PROMPT, HumanMessagePromptTemplate]
)

prompt = chat_prompt.format_prompt(context=context, question=question)

ai_msg = llm.invoke(prompt)


In [21]:
# ===== DISPLAY RESULTS =====
# Show the complete RAG pipeline results for analysis and learning
print("🎯 RAG PIPELINE RESULTS")
print("=" * 50)

print(f"\n❓ ORIGINAL QUESTION:")
print(f"   {question}")

print(f"\n📊 PIPELINE STATISTICS:")
print(f"   • Document chunks retrieved: {len(retrieved_lines_with_distances)}")
print(f"   • Context length: {len(context):,} characters")
print(f"   • Context words: {len(context.split()):,}")
print(f"   • Similarity scores: {[f'{d:.3f}' for _, d in retrieved_lines_with_distances]}")

print(f"\n📝 RETRIEVED CONTEXT SUMMARY:")
for i, (text, score) in enumerate(retrieved_lines_with_distances):
    print(f"   Chunk {i+1} ({score:.3f}): {text[:80]}...")

🎯 RAG PIPELINE RESULTS

❓ ORIGINAL QUESTION:
   What are the challenges of assessing the quality of AI-generated code? What are some strategies for doing this?

📊 PIPELINE STATISTICS:
   • Document chunks retrieved: 3
   • Context length: 1,572 characters
   • Context words: 211
   • Similarity scores: ['0.701', '0.645', '0.628']

📝 RETRIEVED CONTEXT SUMMARY:
   Chunk 1 (0.701): The rise of Large Language Models (LLMs) in software engineering, particularly i...
   Chunk 2 (0.645): The SBC score, along with the reverse-generated requirements, provides actionabl...
   Chunk 3 (0.628): AI-powered code assistants, leveraging the power of Large Language Models (LLMs)...


In [22]:
# ===== CONTEXT ANALYSIS =====
print(f"\n🧠 CONTEXT PROVIDED TO LLM:")
print(f"   Length: {len(context):,} characters ({len(context.split())} words)")
print(f"   Number of chunks: {len(retrieved_lines_with_distances)}")
print(f"\n   First 200 characters:")
print(f"   \"{context[:200]}...\"")

print(f"\n   Last 200 characters:")
print(f"   \"...{context[-200:]}\"")

# Show the context sources
print(f"\n📚 CONTEXT SOURCES:")
for i, (text, score) in enumerate(retrieved_lines_with_distances):
    print(f"   Source {i+1} (similarity: {score:.3f})")
    print(f"      Preview: {text[:80]}...")


🧠 CONTEXT PROVIDED TO LLM:
   Length: 1,572 characters (211 words)
   Number of chunks: 3

   First 200 characters:
   "The rise of Large Language Models (LLMs) in software engineering, particularly in code generation, has garnered significant attention. However, assessing the quality of AI-generated code remains a cha..."

   Last 200 characters:
   "... evaluating the quality of LLM-generated code remains a complex challenge due to the intricacies of programming concepts and syntax, which differ significantly from natural language generation [1, 2]."

📚 CONTEXT SOURCES:
   Source 1 (similarity: 0.701)
      Preview: The rise of Large Language Models (LLMs) in software engineering, particularly i...
   Source 2 (similarity: 0.645)
      Preview: The SBC score, along with the reverse-generated requirements, provides actionabl...
   Source 3 (similarity: 0.628)
      Preview: AI-powered code assistants, leveraging the power of Large Language Models (LLMs)...


In [23]:
# ===== FINAL RESPONSE ANALYSIS =====
print(f"\n🎯 RAG SYSTEM RESPONSE")
print(f"=" * 60)

print(f"\n💬 GENERATED RESPONSE:")
print(f"{ai_msg.content}")

print(f"\n📊 RESPONSE ANALYSIS:")
response_words = len(ai_msg.content.split())
response_chars = len(ai_msg.content)
print(f"   • Response length: {response_chars} characters")
print(f"   • Response words: {response_words}")
print(f"   • Structure: {'Well-structured' if '1.' in ai_msg.content or '•' in ai_msg.content else 'Paragraph format'}")
print(f"   • Addresses both challenges and strategies: {'Yes' if 'challenges' in ai_msg.content.lower() and 'strategies' in ai_msg.content.lower() else 'Partial'}")

print(f"\n✅ RAG PIPELINE COMPLETE!")
print(f"   The system successfully:")
print(f"   • Embedded the user's question")
print(f"   • Retrieved relevant document chunks")
print(f"   • Generated a contextually grounded response")
print(f"   • Provided specific, accurate information from the source document")


🎯 RAG SYSTEM RESPONSE

💬 GENERATED RESPONSE:
The challenges of assessing the quality of AI-generated code include:

1. Complexity of programming tasks
2. Lack of robust evaluation metrics that align with human judgment
3. Inherent differences between programming concepts and syntax and natural language generation

Strategies for assessing AI-generated code quality include:

1. Using the SBC score and reverse-generated requirements for actionable insights
2. Addressing syntactic variations and alternative solutions in generated code
3. Developing evaluation metrics that are specifically designed for code intelligence and verification tasks.

📊 RESPONSE ANALYSIS:
   • Response length: 602 characters
   • Response words: 80
   • Structure: Well-structured
   • Addresses both challenges and strategies: Yes

✅ RAG PIPELINE COMPLETE!
   The system successfully:
   • Embedded the user's question
   • Retrieved relevant document chunks
   • Generated a contextually grounded response
   • Prov

# 🎓 Conclusion: RAG System Complete!

## What We Accomplished

You've successfully built and run a complete RAG (Retrieval-Augmented Generation) system! Here's what we covered:

### 📄 **Document Ingestion**
- Downloaded documents from object storage (MinIO/S3)
- Processed PDF documents using advanced parsing (Docling)
- Performed intelligent document chunking for optimal retrieval

### 🧠 **Text Embeddings**
- Learned about semantic vector representations
- Used SentenceTransformers to generate 384-dimensional embeddings
- Understood how embeddings capture semantic similarity

### 🗃️ **Vector Database**
- Set up Milvus for high-performance vector storage
- Stored embeddings with metadata for efficient retrieval
- Configured search parameters for optimal performance

### 🔍 **Semantic Search**
- Converted queries to embeddings for similarity search
- Retrieved the most relevant document chunks
- Analyzed similarity scores and retrieval quality

### 🤖 **Response Generation**
- Designed effective prompts for contextual responses
- Integrated with Llama 3.2 3B model for generation
- Generated accurate, grounded responses using retrieved context

## Key Takeaways

### RAG Benefits
- **Accuracy**: Responses grounded in specific document content
- **Transparency**: See exactly which sources informed the answer
- **Flexibility**: Easy to update knowledge by changing documents
- **Efficiency**: No need to retrain models for new information

### Technical Insights
- **Embedding Quality**: Choice of embedding model impacts retrieval performance
- **Chunking Strategy**: Proper document segmentation improves context relevance
- **Prompt Engineering**: Well-designed prompts are crucial for quality responses
- **Vector Search**: Semantic similarity enables meaning-based retrieval

## Next Steps

To extend this RAG system, consider:

1. **Multiple Documents**: Expand to handle document collections
2. **Advanced Chunking**: Implement hybrid or semantic chunking strategies
3. **Reranking**: Add reranking models to improve retrieval quality
4. **Evaluation Metrics**: Implement retrieval and generation quality metrics
5. **Production Deployment**: Scale for production with distributed systems
6. **Multi-modal RAG**: Extend to handle images, tables, and other content types

## Learning Resources

- **Vector Databases**: Explore other options like Pinecone, Weaviate, Chroma
- **Embedding Models**: Try domain-specific or larger embedding models
- **LLM Options**: Experiment with different language models and sizes
- **Advanced RAG**: Learn about query expansion, hypothesis verification, and multi-hop reasoning

**Congratulations!** You now understand the fundamentals of building production-ready RAG systems. This knowledge forms the foundation for many modern AI applications that combine retrieval and generation capabilities.
